# Demo Balance de Energía

**Python (Pandas)**
*   Simplicidad
*   Familiaridad
*   Rendimiento para pequeños volumenes
*   Bajo costo (open source)

**Colab**

* Implementa procesos automáticos de respaldo, mediante scripts que sincronicen con Google Drive, además de que permite la mitigación de errores mediante la implementación de notebooks (paso a paso)
* Entorno de desarrollo serverless eficiente, entendible y fácil de manipular
* Completa integración con python

**Escalabilidad**

Integración con Dask: computación y procesamiento de datos distribuída (paralización) para medianos volumenes de datos.





Importe de librerías:

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import re
import os ## Librería del SO para cargar y leer archivos

Definición e inicialización de variables necesarias:

In [ ]:
fecha_inicio="2024-07-09" ## Para que se pueda modificar la fecha del proceso en caso de que se requiera
fecha_fin="2024-07-20"
id_archivo_balance_drive="1EDzz9Ep9VzHwVrHNy9Z_dL5K8D0r4oKv" #ID Archivo con errores!!
id_archivo_balance_drive_cor="1gsiY85tRa5_3QLtjRvEAz403tTgkSLDn" #ID Archivo sin error
api_key_1="lbmgOxpdzM9ffbXmvF3UyPhxwEHFf53wFSDk8ddqEQeor9eXetbGiYqlUkKPHIBL"
api_key_2="QlgVYk5LeVCWCD6BoNoWFkn1lhtUC3vPuFpWBuNkpRo6R4trrNnyqux6LTTn8I4n"

# Extracción

**Para procesar el archivo con errores en sus valores:**

In [ ]:
download_url = f'https://drive.google.com/uc?export=download&id={id_archivo_balance_drive}'
local_file_path = '/tmp/balances3.xlsx'

## "response" es un objeto que contiene los datos en binario que se desean escribir en el archivo
response = requests.get(download_url)
with open(local_file_path, 'wb') as file: ## Abre un archivo en modo escritura binaria ('wb'). El gestor (context manager) asegura que el archivo se cierre automáticamente cuando se salga del bloque "with"
    file.write(response.content) ## Este método escribe datos en binario en el archivo abierto "file".

**Para procesar el archivo sin errores en sus valores:**

In [ ]:
download_url_cor = f'https://drive.google.com/uc?export=download&id={id_archivo_balance_drive_cor}'
local_file_path = '/tmp/balances4.xlsx'
response = requests.get(download_url_cor)
with open(local_file_path, 'wb') as file:
    file.write(response.content)

Capacidad real de generadoras

In [ ]:
dfArchivoCapacidad = pd.read_excel(local_file_path,engine='openpyxl')
dfArchivoCapacidad.head(10).style

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,nan,Programa de Generación por Planta,nan,nan,nan,nan
1,nan,Empresa Generadora ACME S.A. E.S.P.,nan,nan,nan,nan
2,nan,Despacho Real 10/07/2024,nan,nan,nan,nan
3,nan,nan,nan,nan,nan,nan
4,nan,FECHA,PLANTA,GENERADOR,CAPACIDAD (Kwh),CODIGO
5,nan,2024-07-10 00:00:00,TERMOZIPA,2,33000,ZPA2
6,nan,2024-07-10 01:00:00,TERMOZIPA,2,33000,ZPA2
7,nan,2024-07-10 02:00:00,TERMOZIPA,2,33000,ZPA2
8,nan,2024-07-10 03:00:00,TERMOZIPA,2,33000,ZPA2
9,nan,2024-07-10 04:00:00,TERMOZIPA,2,20834,ZPA2


Para obtener los precios de energía desde la bolsa, consumiendo la API de XM

In [ ]:
## Para obtener desde el simem los precios. el datasetId al final de la url hay que buscarlo en simem Precios
url = f"https://www.simem.co/backend-files/api/PublicData?startDate={fecha_inicio}&endDate={fecha_fin}&datasetId=96D56E"

response = requests.get(url)
json_precios = json.loads(response.text)

In [ ]:
dfPreciosBolsa = pd.DataFrame(json_precios['result']['records'])
dfPreciosBolsa.head(10).style


,CodigoVariable,Fecha,CodigoDuracion,UnidadMedida,Version,Valor
0,PPBOGReal,2024-07-20,P1D,COP/kWh,TXF,321.583900
1,PPBO,2024-07-20,P1D,COP/kWh,TXF,322.321900
2,PPBO,2024-07-20,P1D,COP/kWh,TX2,316.251600
3,PPBOGReal,2024-07-20,P1D,COP/kWh,TX2,315.504300
4,PPBOGReal,2024-07-20,P1D,COP/kWh,TX1,315.454800
5,PPBOGReal,2024-07-20,P1D,COP/kWh,TXR,321.582000
6,PPBO,2024-07-20,P1D,COP/kWh,TXR,322.323700
7,PPBOGReal,2024-07-19,P1D,COP/kWh,TX1,315.522600
8,PPBOGReal,2024-07-19,P1D,COP/kWh,TXF,321.599900
9,PPBO,2024-07-19,P1D,COP/kWh,TXF,319.463500


In [ ]:
## Para obtener desde SIMEM el compromiso o los despachos por unidad.
url = f"https://www.simem.co/backend-files/api/PublicData?startdate={fecha_inicio}&enddate={fecha_fin}&datasetId=ff027b"
response = requests.get(url)
json_precios = json.loads(response.text)

In [ ]:
dfDespachosUnidades = pd.DataFrame(json_precios['result']['records'])
dfDespachosUnidades.head(10).style

,FechaHora,CodigoDuracion,CodigoPlanta,Valor
0,2024-07-20 13:00:00,PT1H,SBA1,0.000000
1,2024-07-20 13:00:00,PT1H,SFC1,370.000000
2,2024-07-20 13:00:00,PT1H,SJM1,0.000000
3,2024-07-20 13:00:00,PT1H,SJN1,2700.000000
4,2024-07-20 13:00:00,PT1H,SJOG,270.000000
5,2024-07-20 13:00:00,PT1H,SLV1,350.000000
6,2024-07-20 13:00:00,PT1H,SLVJ,125000.000000
7,2024-07-20 13:00:00,PT1H,SMI1,47000.000000
8,2024-07-20 13:00:00,PT1H,SNC1,800.000000
9,2024-07-20 13:00:00,PT1H,SNCR,0.000000


# Transformación

Ahora, es necesario filtrar la columna código de planta en el DF de despachos para que permanezcan solo los registros que contengan ZPA2, ZPA3, ZPA4, ZPA5, GVIO, QUI1, CHVR

In [ ]:
listaPlantas = ['ZPA2','ZPA3','ZPA4','ZPA5','GVIO','QUI1']

In [ ]:
#Función para filtrar un dataframe por código planta.
#Recibe el dataframe implicado y una lista que contiene los código planta que se quiere
#Retorna el dataframe con los registros cuyo valor es True (filtro interno con el isin)

def filtroCodigoPlanta(df, codigos):
    return df[df['CodigoPlanta'].isin(codigos)]


In [ ]:
dfCapacidadDespachosAcme = filtroCodigoPlanta(dfDespachosUnidades, listaPlantas)

# Resetear los índices del dataframe
dfCapacidadDespachosAcme = dfCapacidadDespachosAcme.reset_index(drop=True)

dfCapacidadDespachosAcme.head(10).style

,FechaHora,CodigoDuracion,CodigoPlanta,Valor
0,2024-07-20 13:00:00,PT1H,QUI1,79000.000000
1,2024-07-20 14:00:00,PT1H,QUI1,79000.000000
2,2024-07-20 14:00:00,PT1H,GVIO,573380.000000
3,2024-07-20 12:00:00,PT1H,QUI1,79000.000000
4,2024-07-20 13:00:00,PT1H,GVIO,121450.000000
5,2024-07-20 12:00:00,PT1H,ZPA2,0.000000
6,2024-07-20 12:00:00,PT1H,ZPA3,48000.000000
7,2024-07-20 12:00:00,PT1H,ZPA4,48000.000000
8,2024-07-20 12:00:00,PT1H,ZPA5,0.000000
9,2024-07-20 15:00:00,PT1H,ZPA2,0.000000


A continuación se renombran las columnas del dataframe de la capacidad y se nomalizan los datos a tipo string

In [ ]:
# Renombramiento de columnas
colRenom = {'Unnamed: 0':'Useless',
            'Unnamed: 1':'Fecha',
            'Unnamed: 2':'Planta',
            'Unnamed: 3':'Generador',
            'Unnamed: 4':'Capacidad',
            'Unnamed: 5':'Codigo'}

dfArchivoCapacidad = dfArchivoCapacidad.rename(columns=colRenom).astype(str)
dfArchivoCapacidad.head(10).style

,Useless,Fecha,Planta,Generador,Capacidad,Codigo
0,nan,Programa de Generación por Planta,nan,nan,nan,nan
1,nan,Empresa Generadora ACME S.A. E.S.P.,nan,nan,nan,nan
2,nan,Despacho Real 10/07/2024,nan,nan,nan,nan
3,nan,nan,nan,nan,nan,nan
4,nan,FECHA,PLANTA,GENERADOR,CAPACIDAD (Kwh),CODIGO
5,nan,2024-07-10 00:00:00,TERMOZIPA,2,33000,ZPA2
6,nan,2024-07-10 01:00:00,TERMOZIPA,2,33000,ZPA2
7,nan,2024-07-10 02:00:00,TERMOZIPA,2,33000,ZPA2
8,nan,2024-07-10 03:00:00,TERMOZIPA,2,33000,ZPA2
9,nan,2024-07-10 04:00:00,TERMOZIPA,2,20834,ZPA2


Se elimina la información innecesaria del archivo de capacidad real

In [ ]:
# Eliminar filas innecesarias antes de la fila 4 (donde están los encabezados reales)
dfArchivoCapacidad = dfArchivoCapacidad.drop([0, 1, 2, 3, 4])

# Eliminar la columna 'Useless' que no es necesaria
dfArchivoCapacidad = dfArchivoCapacidad.drop(columns=['Useless'])

# Resetear los índices del dataframe
dfArchivoCapacidad = dfArchivoCapacidad.reset_index(drop=True)

# Mostrar el dataframe limpio
dfArchivoCapacidad.head(10).style

,Fecha,Planta,Generador,Capacidad,Codigo
0,2024-07-10 00:00:00,TERMOZIPA,2,33000,ZPA2
1,2024-07-10 01:00:00,TERMOZIPA,2,33000,ZPA2
2,2024-07-10 02:00:00,TERMOZIPA,2,33000,ZPA2
3,2024-07-10 03:00:00,TERMOZIPA,2,33000,ZPA2
4,2024-07-10 04:00:00,TERMOZIPA,2,20834,ZPA2
5,2024-07-10 05:00:00,TERMOZIPA,2,21626,ZPA2
6,2024-07-10 06:00:00,TERMOZIPA,2,26030,ZPA2
7,2024-07-10 07:00:00,TERMOZIPA,2,33000,ZPA2
8,2024-07-10 08:00:00,TERMOZIPA,2,33000,ZPA2
9,2024-07-10 09:00:00,TERMOZIPA,2,33000,ZPA2


Se hace validación de formatos y verificación de los valores contenidos en el dataframe de la capacidad real de las generadoras.

In [27]:
def validar_fecha(fecha):
    patron = r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$'
    return bool(re.match(patron, fecha))

def validar_generador(generador):
    try:
        int(generador)
        return True
    except ValueError:
        return False

def validar_capacidad(capacidad):
    try:
        int(capacidad)
        if int(capacidad) > 0: return True
        else: return False
    except ValueError:
        return False



# Se aplican las funciones de validación a los valores de las columnas correspondientes

dfArchivoCapacidad['Fecha_valida'] = dfArchivoCapacidad['Fecha'].apply(validar_fecha)
dfArchivoCapacidad['Generador_valido'] = dfArchivoCapacidad['Generador'].apply(validar_generador)
dfArchivoCapacidad['Capacidad_valida'] = dfArchivoCapacidad['Capacidad'].apply(validar_capacidad)

# Se filtran las filas que no cumplen con el formato de fecha y las condiciones del generador
fechas_invalidas = dfArchivoCapacidad[~dfArchivoCapacidad['Fecha_valida']]
generadores_invalidos = dfArchivoCapacidad[~dfArchivoCapacidad['Generador_valido']]
capacidades_invalidas = dfArchivoCapacidad[~dfArchivoCapacidad['Capacidad_valida']]

# Captura de error y reporte al usuario
if not fechas_invalidas.empty:
    print("Fechas con formato incorrecto. Por favor revisar el archivo")
    print(fechas_invalidas)

elif not generadores_invalidos.empty:
    print("Generadores con formato incorrecto. Por favor revisar el archivo")
    print(generadores_invalidos)

elif not capacidades_invalidas.empty:
    print("Capacidades invalidas. Por favor revisar el archivo")
    print(capacidades_invalidas)

else:

    # Se filtran las filas que cumplen con todas las validaciones
    df_validado = dfArchivoCapacidad[(dfArchivoCapacidad['Fecha_valida']) &
                                    (dfArchivoCapacidad['Generador_valido']) &
                                    (dfArchivoCapacidad['Fecha'] >= fecha_inicio) &
                                    (dfArchivoCapacidad['Fecha'] <= fecha_fin)]

    # Se procede a dividir la columna Fecha y crear el dataframe limpio
    df_validado['Año'] = pd.to_datetime(dfArchivoCapacidad['Fecha']).dt.year
    df_validado['Mes'] = pd.to_datetime(dfArchivoCapacidad['Fecha']).dt.month
    df_validado['Día'] = pd.to_datetime(dfArchivoCapacidad['Fecha']).dt.day
    df_validado['Hora'] = pd.to_datetime(dfArchivoCapacidad['Fecha']).dt.hour

    # Se eliminan las columnas de validación
    df_validado = df_validado.drop(columns=['Fecha_valida', 'Generador_valido'])

    # Selección de las columnas necesarias
    dfArchivoCapacidadLimpio = df_validado[['Año', 'Mes', 'Día', 'Hora', 'Codigo', 'Capacidad']]

    print(dfArchivoCapacidadLimpio)

      Año  Mes  Día  Hora Codigo Capacidad
0    2024    7   10     0   ZPA2     33000
1    2024    7   10     1   ZPA2     33000
2    2024    7   10     2   ZPA2     33000
3    2024    7   10     3   ZPA2     33000
4    2024    7   10     4   ZPA2     20834
..    ...  ...  ...   ...    ...       ...
139  2024    7   10    19   QUI1    284650
140  2024    7   10    20   QUI1    276992
141  2024    7   10    21   QUI1    340000
142  2024    7   10    22   QUI1    340000
143  2024    7   10    23   QUI1    340000

[144 rows x 6 columns]


Separación de la fecha, renombramiento y extracción de las columnas necesarias en el dataframe de la capacidad que XM solicita

In [22]:
dfCapacidadDespachosAcme['dia_despacho'] = pd.to_datetime(dfCapacidadDespachosAcme['FechaHora']).dt.day
dfCapacidadDespachosAcme['mes_despacho'] = pd.to_datetime(dfCapacidadDespachosAcme['FechaHora']).dt.month
dfCapacidadDespachosAcme['anio_despacho'] = pd.to_datetime(dfCapacidadDespachosAcme['FechaHora']).dt.year
dfCapacidadDespachosAcme['hora_despacho'] = pd.to_datetime(dfCapacidadDespachosAcme['FechaHora']).dt.hour

dfCapacidadDespachosAcme = dfCapacidadDespachosAcme.rename(columns={'Valor':'capacidad', 'CodigoPlanta':'codigo'})
dfDespachosAcmeTransformado = dfCapacidadDespachosAcme[['dia_despacho', 'mes_despacho', 'anio_despacho', 'hora_despacho', 'capacidad', 'codigo']]


dfDespachosAcmeTransformado.head(10).style

,dia_despacho,mes_despacho,anio_despacho,hora_despacho,capacidad,codigo
0,20,7,2024,13,79000.000000,QUI1
1,20,7,2024,14,79000.000000,QUI1
2,20,7,2024,14,573380.000000,GVIO
3,20,7,2024,12,79000.000000,QUI1
4,20,7,2024,13,121450.000000,GVIO
5,20,7,2024,12,0.000000,ZPA2
6,20,7,2024,12,48000.000000,ZPA3
7,20,7,2024,12,48000.000000,ZPA4
8,20,7,2024,12,0.000000,ZPA5
9,20,7,2024,15,0.000000,ZPA2


Cálculo del balance de energía

In [28]:
# Une los DataFrames por las columnas especificadas
dfFusionado = pd.merge(dfArchivoCapacidadLimpio, dfDespachosAcmeTransformado,
                    left_on=['Codigo', 'Año', 'Mes', 'Día', 'Hora'],
                    right_on=['codigo', 'anio_despacho', 'mes_despacho', 'dia_despacho', 'hora_despacho'])

# Calcular el balance disponible horario
dfFusionado['balance_disponible_horario'] = dfFusionado['Capacidad'].astype(int) - dfFusionado['capacidad'].astype(int)

# Agrupa, calcula la suma y resetea el index
dfBalanceConsolidado = dfFusionado.groupby(['Año', 'Mes', 'Día', 'Codigo'])['balance_disponible_horario'].sum().reset_index()
dfBalanceConsolidado.rename(columns={'balance_disponible_horario': 'consolidado_planta'}, inplace=True)

dfBalanceConsolidado.head(10).style

,Año,Mes,Día,Codigo,consolidado_planta
0,2024,7,10,GVIO,11506280
1,2024,7,10,QUI1,3607414
2,2024,7,10,ZPA2,201706
3,2024,7,10,ZPA3,-280652
4,2024,7,10,ZPA4,-203312
5,2024,7,10,ZPA5,-362060


Cálculo del precio del balance y se agrega la columna de la operación a realizar según el resultado

In [38]:
# Filtra los precios de bolsa por las condiciones especificadas. Los copy es para trabajar sobre copias de los DF
dfPreciosFiltrados = dfPreciosBolsa[(dfPreciosBolsa['CodigoVariable'] == 'PPBOGReal') & (dfPreciosBolsa['Version'] == 'TXR')].copy()

# Crea columnas de año, mes y día en dfPreciosFiltrados
dfPreciosFiltrados.loc[:, 'Año'] = pd.to_datetime(dfPreciosFiltrados['Fecha']).dt.year
dfPreciosFiltrados.loc[:, 'Mes'] = pd.to_datetime(dfPreciosFiltrados['Fecha']).dt.month
dfPreciosFiltrados.loc[:, 'Día'] = pd.to_datetime(dfPreciosFiltrados['Fecha']).dt.day

# Une los DataFrames por las columnas año, mes y dia
dfUnido = pd.merge(dfBalanceConsolidado, dfPreciosFiltrados, on=['Año', 'Mes', 'Día'], how='inner')

# Calcular Compromisos_MCOP
dfUnido['Compromisos_MCOP'] = (dfUnido['consolidado_planta'] * dfUnido['Valor']) / 1000

# Seleccionar las columnas deseadas
dfReporteCompraVentaEnergiaAcme = dfUnido[['Año', 'Mes', 'Día', 'Codigo', 'consolidado_planta', 'Compromisos_MCOP']].copy()

# Crea la nueva columna de Operación
# Si los compromisos son menores a 0, hay que comprar energía
dfReporteCompraVentaEnergiaAcme['Operacion'] = np.where(dfReporteCompraVentaEnergiaAcme['Compromisos_MCOP'] < 0, 'Comprar', 'Vender')

dfReporteCompraVentaEnergiaAcme.rename(columns={'Año': 'Anio', 'Día': 'Dia'}, inplace=True)

dfReporteCompraVentaEnergiaAcme.head(10).style


,Anio,Mes,Dia,Codigo,consolidado_planta,Compromisos_MCOP,Operacion
0,2024,7,10,GVIO,11506280,2528576.368936,Vender
1,2024,7,10,QUI1,3607414,792751.592467,Vender
2,2024,7,10,ZPA2,201706,44326.144077,Vender
3,2024,7,10,ZPA3,-280652,-61675.017042,Comprar
4,2024,7,10,ZPA4,-203312,-44679.072534,Comprar
5,2024,7,10,ZPA5,-362060,-79564.929772,Comprar


# Carga

En las siguientes líneas de código se almacena localmente en el notebook una carpeta que contendrá el archivo

In [39]:
ruta_carpeta = 'tmp'
os.makedirs(ruta_carpeta, exist_ok=True)
dfReporteCompraVentaEnergiaAcme.to_csv(f'{ruta_carpeta}//dfReporteCompraVentaEnergiaAcme.csv', index=False)

Inicio de sesión en la plataforma con la API y obtención del token para cargar el archivo

In [40]:
file_upload_access_token = ""
file_upload_account_id = ""
params = {'key1': api_key_1, 'key2': api_key_2}
response=requests.get("https://fastupload.io/api/v2/authorize", params)
json_response = json.loads(response.text)
try:
  file_upload_access_token = json_response["data"]["access_token"]
  file_upload_account_id = json_response["data"]["account_id"]
except:
  print("Error autenticando y autorizando en el servicio remoto de carga de archivos.")

In [41]:
print(file_upload_access_token)

WEfQKPpxfklwF2PmuMLLh3TaGYqTKWYQaQiL8WM9Ttiwo189rHG349YqMoHcRMyN6qopXSopYPfZnIqLXiYvWD8DjvIBctVkj47z2h3H859X4brN8nb185zuWCUB8c3p


Finalmente la conexión a la API por medio de las Key y carga del archivo (post)

In [42]:
upload_folder_id = ""
json_response=""
with open("tmp/dfReporteCompraVentaEnergiaAcme.csv", "rb") as archivo:
  files = {"upload_file": (archivo.name, archivo)}  # Create a dictionary for the file upload

  data = {
    "access_token": file_upload_access_token,
    "account_id": file_upload_account_id,
    "folder_id": upload_folder_id  # Include folder_id if provided
  }

  response = requests.post("https://fastupload.io/api/v2/file/upload", files=files, data=data)

print(response.text)

{"response":"File uploaded","data":[{"name":"dfReporteCompraVentaEnergiaAcme (3).csv","size":342,"type":"application\/octet-stream","error":null,"url":"https:\/\/fastupload.io\/eda68a5857c3465e","delete_url":"https:\/\/fastupload.io\/eda68a5857c3465e~d?c8f6767169fe76f30b47c7d4a7ed7321","info_url":"https:\/\/fastupload.io\/eda68a5857c3465e~i","delete_type":"DELETE","delete_hash":"c8f6767169fe76f30b47c7d4a7ed7321","hash":"94f88bd605991fa72ea7751ae465407a","stats_url":"https:\/\/fastupload.io\/eda68a5857c3465e~s","short_url":"eda68a5857c3465e","file_id":"665876","unique_hash":"b93530f4e0ec1da81d4c1b666da050a4552a803c524d7304339686355c0a04d8","url_html":"&lt;a href=&quot;https:\/\/fastupload.io\/eda68a5857c3465e&quot; target=&quot;_blank&quot; title=&quot;View image on Fastupload.io&quot;&gt;View dfReporteCompraVentaEnergiaAcme (3).csv on Fastupload.io&lt;\/a&gt;","url_bbcode":"[url]https:\/\/fastupload.io\/eda68a5857c3465e[\/url]","success_result_html":""}],"_status":"success","_datetime"